In [1]:
# ! pip install pandasql

### Import Necessary Libraries

In [2]:
import os
import pandas as pd
from pandasql import sqldf
from datetime import datetime, timedelta

curr_date = datetime.now().strftime('%d-%m-%Y')
prev_day = (datetime.now() - timedelta(days=1)).strftime('%d-%m-%Y')

print("Current date:", curr_date)
print("Previous day:", prev_day)

Current date: 20-06-2024
Previous day: 19-06-2024


In [3]:
output_dir = "../Output"
os.makedirs(output_dir, exist_ok=True)

### Import Data Files

In [4]:
in_path = os.path.join(output_dir, f"Inwards successful_{prev_day}.csv")

if os.path.exists(in_path):
    in_df = pd.read_csv(in_path)
    # display(in_df)
    print("Inwards File loaded Successfully!")
else:
    print("Inwards file does not exist in the specified path.")

out_path = os.path.join(output_dir, f"Outwards successful_{prev_day}.csv")

if os.path.exists(out_path):
    out_df = pd.read_csv(out_path)
    # display(in_df)
    print("Outwards File loaded Successfully!")
else:
    print("Outwards file does not exist in the specified path.")

Inwards File loaded Successfully!


C:\Users\ijohnson\AppData\Local\Temp\ipykernel_11380\2471884690.py:13: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  out_df = pd.read_csv(out_path)


Outwards File loaded Successfully!


### Count the number of records in the files

In [5]:
inwards = "SELECT count(1) knt from in_df"
inwards_df = sqldf(inwards, locals())
print("Inwards Count is Ready!!!")

outwards = "SELECT count(1) knt from out_df"
outwards_df = sqldf(outwards, locals())
print("Outwards Count is Ready!!!")

Inwards Count is Ready!!!
Outwards Count is Ready!!!


In [6]:
print(outwards_df)
print(inwards_df)

      knt
0  299837
      knt
0  455459


### Data Transformation and Filtering

In [7]:
# EBILLS DATA
ebills = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('EBILLSPAY', 'NIBSS_EASY_PAY')
"""
ebills_df = sqldf(ebills, locals())
print("Ebills Data is Ready!!!")

# PAYABLE DATA
payable = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('CMMS', 'NIBSS_NIP_POS','NIP','NIP DISPUTE','NIBSS_NIP_GSI','NIBSS_NIP_EBILLSPAY','NIP DISPUTE')
and CHANNEL not in ('ATM','MOBILE PHONE')
UNION
SELECT a.*, 'INWARDS' AS fileType FROM in_df a WHERE category in ('NIP REVERSAL')
"""
payable_df = sqldf(payable, locals())
print("Payable Data is Ready!!!")

# PAYABLE DATA
bulk_payable = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('CMMS', 'NIBSS_NIP_POS','NIP','NIP DISPUTE','NIBSS_NIP_GSI','NIBSS_NIP_EBILLSPAY','NIP DISPUTE')
UNION
SELECT a.*, 'INWARDS' AS fileType FROM in_df a WHERE category in ('NIP REVERSAL')
"""
bulk_payable_df = sqldf(bulk_payable, locals())
print("Bulk payable Data is Ready!!!")

# RECEIVABLE DATA
receivable = """
SELECT a.*, 'INWARDS' AS fileType FROM in_df a WHERE category in ('CMMS', 'EBILLSPAY', 'NIBSS_EASY_PAY', 'NIBSS_NIP_POS', 'NIP', 'NIBSS_NIP_GSI', 'NIBSS_NIP_EBILLSPAY', 'NIP DISPUTE','NQR_MERCHANT')
UNION
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('NIP REVERSAL')
"""
receivable_df = sqldf(receivable, locals())
print("Receivable Data is Ready!!!")

# ATM DATA
atm = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('NIP')
and CHANNEL in ('ATM')
"""
atm_df = sqldf(atm, locals())
print("ATM Data is Ready!!!")

# MOBILE DATA
mobile = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('NIP')
and CHANNEL in ('MOBILE PHONE') and SUBSTR(`PAYMENT REFERENCE`, 2) NOT LIKE 'A24%'
"""
mobile_df = sqldf(mobile, locals())
print("Mobile Data is Ready!!!")

# AGENCY DATA
agency = """
SELECT a.*, 'OUTWARDS' AS fileType FROM out_df a WHERE category in ('NIP')
and SUBSTR(`PAYMENT REFERENCE`, 2) LIKE 'A24%'
"""
agency_df = sqldf(agency, locals())
print("Agency Data is Ready!!!")

Ebills Data is Ready!!!
Payable Data is Ready!!!
Bulk payable Data is Ready!!!
Receivable Data is Ready!!!
ATM Data is Ready!!!
Mobile Data is Ready!!!
Agency Data is Ready!!!


In [8]:
display(ebills_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,1,THIRD-PARTY PAYMENT PLATFORM,'999999240619001356323764028923,Fund Transfer (Direct Credit),Approved or Completed Successfully,50.00,2024-06-19 00:14:00,ECOBANK,ANIFOWOSE TAWAKALITU KIKELOMO,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-3511260319,'NIPMINI/110006240619001355000000979454/PSTK/T...,'323764028923,NIBSS_EASY_PAY,ECOBANK_20240619_055959,OUTWARDS
1,2,THIRD-PARTY PAYMENT PLATFORM,'999999240619024307618012585918,Fund Transfer (Direct Credit),Approved or Completed Successfully,2000.00,2024-06-19 02:43:10,ECOBANK,ADENIYAN ADETUTU BLESSING,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-4263005818,'NIPMINI/110006240619024306000000979983/PSTK/T...,'618012585918,NIBSS_EASY_PAY,ECOBANK_20240619_055959,OUTWARDS
2,3,THIRD-PARTY PAYMENT PLATFORM,'999999240619024544356670058686,Fund Transfer (Direct Credit),Approved or Completed Successfully,1000.00,2024-06-19 02:45:47,ECOBANK,OMOREGBEE WINNIE EDUGIE,WEMA BANK,PAYSTACK DIRECT DEBIT COLLECTION ACCOUNT,1300011066,TRF-PAYSTACK-DIRECT-DEBIT-4400000562,'NIPMINI/110006240619024543000000980001/PSTK/T...,'356670058686,NIBSS_EASY_PAY,ECOBANK_20240619_055959,OUTWARDS
3,4,MOBILE PHONE,'999999240619010040790943375062,Fund Transfer (Direct Credit),Approved or Completed Successfully,812.03,2024-06-19 01:00:44,ECOBANK,ABRAHAM DESTINY SIMON,ZENITH BANK,NEWEDGE FINANCE LID NIP,1223458380,Payment to 1223458380 from 2341243998,'NIPMINI/050004240619010040144127660041/xc7234...,'790943375062,NIBSS_EASY_PAY,ECOBANK_20240619_055959,OUTWARDS
4,5,MOBILE PHONE,'999999240619010043479453851645,Fund Transfer (Direct Credit),Approved or Completed Successfully,13207.41,2024-06-19 01:00:45,ECOBANK,NATHAN TSAGA,ZENITH BANK,NEWEDGE FINANCE LID NIP,1223458380,Payment to 1223458380 from 2621202369,'NIPMINI/050004240619010042153749397533/xc7234...,'479453851645,NIBSS_EASY_PAY,ECOBANK_20240619_055959,OUTWARDS


In [9]:
display(payable_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,872,INTERNET BANKING,'090134240619053833727993956375,Fund Transfer (Direct Credit),Approved or Completed Successfully,2000.0,2024-06-19 05:37:57,ACCION MFB,ONYEBUCHI IKECHUKWU OGBONNA,ACCESS BANK,RAPHEAL EBUKA UDOJI,69255880,TRF TO RAPHEAL EBUKA UDOJI /JUNE JULY ELECTRIC...,'FT24171M3D6W,'727993956375,NIP,ECOBANK_20240619_055959,OUTWARDS
1,1827,INTERNET BANKING,'090134240619053846691367487290,Fund Transfer (Direct Credit),Approved or Completed Successfully,360000.0,2024-06-19 05:38:11,ACCION MFB,ANIH HYCENT,FIRST BANK,HYCENTH ANIH GLOBAL ENTERPRISES,2040795049,TRF TO HYCENTH ANIH GLOBAL ENTERPRISES/GOODS,'FT24171WD6QC,'691367487290,NIP,ECOBANK_20240619_055959,OUTWARDS
2,1828,INTERNET BANKING,'090134240619055913505055907178,Fund Transfer (Direct Credit),Approved or Completed Successfully,350000.0,2024-06-19 05:58:37,ACCION MFB,ANIH HYCENT,FIRST BANK,HYCENTH ANIH GLOBAL ENTERPRISES,2040795049,TRF TO HYCENTH ANIH GLOBAL ENTERPRISES/GOODS,'FT24171WTPK2,'505055907178,NIP,ECOBANK_20240619_055959,OUTWARDS
3,2318,INTERNET BANKING,'090134240619045859117315779514,Fund Transfer (Direct Credit),Approved or Completed Successfully,20000.0,2024-06-19 04:58:26,ACCION MFB,OGUNTAYO WASIU KAYODE,FCMB,DARAMOLA MOJISOLA OLUWAFERANMI,6181989016,TRF TO DARAMOLA MOJISOLA OLUWAFERANMI/OK,'FT24171VFP4Y,'117315779514,NIP,ECOBANK_20240619_055959,OUTWARDS
4,2902,INTERNET BANKING,'090134240619001153166131872697,Fund Transfer (Direct Credit),Approved or Completed Successfully,50000.0,2024-06-19 00:11:18,ACCION MFB,IKPEFUA ERIC EBAKOR,GTBANK,IKPEFUA ERIC EBAKOR,170702761,TRF TO IKPEFUA ERIC EBAKOR/000013/ TRF IFO IKP...,'FT24171NNQNJ,'166131872697,NIP,ECOBANK_20240619_055959,OUTWARDS


In [10]:
display(bulk_payable_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,22,MOBILE PHONE,'000010240618230006111383882699,Fund Transfer (Direct Credit),Approved or Completed Successfully,10400.0,2024-06-19 00:00:11,ECOBANK,SOLOMON EFFIONG,ACCESS BANK,USORO UDUAK FRANCIS,29331278,REFNGA1400692133 BO SOLOMON EFFIONG IFO USORO UDU,'NGA1400692133,'111383882699,NIP,ECOBANK_20240619_055959,OUTWARDS
1,23,MOBILE PHONE,'000010240618230027111032687235,Fund Transfer (Direct Credit),Approved or Completed Successfully,15000.0,2024-06-19 00:00:30,ECOBANK,ndukwe ndukwe,ACCESS BANK,NNENNA NDUKWE,103834941,REFNGA1400692260 BO ndukwe ndukwe IFO NNENNA NDUK,'NGA1400692260,'111032687235,NIP,ECOBANK_20240619_055959,OUTWARDS
2,24,MOBILE PHONE,'000010240618230033111065196485,Fund Transfer (Direct Credit),Approved or Completed Successfully,112000.0,2024-06-19 00:00:33,ECOBANK,DAISI OLOYEDE,ACCESS BANK,CLEMENT CHIBUEZE OKEKE,1446522277,REF25005973944 BO DAISI OLOYEDE IFO CLEMENT CHIBU,'25005973944,'111065196485,NIP,ECOBANK_20240619_055959,OUTWARDS
3,25,MOBILE PHONE,'000010240618230033111218229745,Fund Transfer (Direct Credit),Approved or Completed Successfully,5000.0,2024-06-19 00:00:36,ECOBANK,OCHUKO IMIRHE,ACCESS BANK,GRACE CHAPEL,1686854062,REFNGA1400692300 BO OCHUKO IMIRHE IFO GRACE CHAPE,'NGA1400692300,'111218229745,NIP,ECOBANK_20240619_055959,OUTWARDS
4,26,MOBILE PHONE,'000010240618230107111366137289,Fund Transfer (Direct Credit),Approved or Completed Successfully,5000.0,2024-06-19 00:01:10,ECOBANK,okoi obeten,ACCESS BANK,COMFORT INI HANSON,1643471318,REFNGA1400692523 BO okoi obeten IFO COMFORT INI H,'NGA1400692523,'111366137289,NIP,ECOBANK_20240619_055959,OUTWARDS


In [11]:
display(receivable_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,1,BANK TELLER,'999999240619021321367936388904,Fund Transfer (Direct Credit),Approved or Completed Successfully,20000.0,2024-06-19 02:13:24,STERLING BANK,LAVENDER MICROFINANCE BANK,ECOBANK,CAC COVENANT OF MERCY,2080047985,125931383876,'NIPMINI/090271240619021320401101354474/090271...,'367936388904,NIBSS_EASY_PAY,ECOBANK_20240619_055959,INWARDS
1,2,BANK TELLER,'999999240619000028337658960126,Fund Transfer (Direct Credit),Approved or Completed Successfully,15000.0,2024-06-19 00:00:29,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,KPODEZIAHOU COMLAN ANGELOT,5051108434,BET9JA WITHDRAWAL,'NIPMINI/090601240619000028572456672734/382c57...,'337658960126,NIBSS_EASY_PAY,ECOBANK_20240619_055959,INWARDS
2,3,BANK TELLER,'999999240619000154355588218988,Fund Transfer (Direct Credit),Approved or Completed Successfully,2000.0,2024-06-19 00:01:54,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,JOSEPH EBIEGBERI SYLVESTER,201081300,BET9JA WITHDRAWAL,'NIPMINI/090601240619000153672126457553/9619e5...,'355588218988,NIBSS_EASY_PAY,ECOBANK_20240619_055959,INWARDS
3,4,BANK TELLER,'999999240619000228323941512752,Fund Transfer (Direct Credit),Approved or Completed Successfully,7000.0,2024-06-19 00:02:29,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,ANGALABIRI RICHARD,3591020160,BET9JA WITHDRAWAL,'NIPMINI/090601240619000228574733436214/d19c91...,'323941512752,NIBSS_EASY_PAY,ECOBANK_20240619_055959,INWARDS
4,5,BANK TELLER,'999999240619000415251476697809,Fund Transfer (Direct Credit),Approved or Completed Successfully,100000.0,2024-06-19 00:04:16,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,OGODU ROMANUS,2543009659,BET9JA WITHDRAWAL,'NIPMINI/090601240619000415775788624534/4f75d0...,'251476697809,NIBSS_EASY_PAY,ECOBANK_20240619_055959,INWARDS


In [12]:
display(atm_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,11333,ATM,'000010240619085441061941118577,Fund Transfer (Direct Credit),Approved or Completed Successfully,20000.0,2024-06-19 08:58:07,ECOBANK,ORUMA DUPE AYOD,FIDELITY BANK,DUPE AYODEJI ORUMA,6975490528,TRANSFER TO DUPE AYODEJI ORUMA FROM ORUMA DUPE...,'000204818556,'061941118577,NIP,ECOBANK_20240619_085959,OUTWARDS
1,11448,ATM,'000010240619060710061910206542,Fund Transfer (Direct Credit),Approved or Completed Successfully,18000.0,2024-06-19 06:10:27,ECOBANK,UNA SUNDAY GODWIN,FIRST BANK,NWALI RUTH NNENNA,3154428432,TRANSFER TO NWALI RUTH NNENNA FROM UNA SUNDAY ...,'000204813553,'061910206542,NIP,ECOBANK_20240619_085959,OUTWARDS
2,11516,ATM,'000010240619061413061913582612,Fund Transfer (Direct Credit),Approved or Completed Successfully,50000.0,2024-06-19 06:17:28,ECOBANK,OGIRI CHARITY,FIRST BANK,OGIRI ONYEJEGBA TITUS,3064089945,TRANSFER TO OGIRI ONYEJEGBA TITUS FROM OGIRI C...,'000204813617,'061913582612,NIP,ECOBANK_20240619_085959,OUTWARDS
3,11644,ATM,'000010240619062539061939204142,Fund Transfer (Direct Credit),Approved or Completed Successfully,20000.0,2024-06-19 06:28:56,ECOBANK,RAHEEM ABDULAZEEZ ABIODUN,FIRST BANK,LAMINA BUSOLA RAMOTA,3161961944,TRANSFER TO LAMINA BUSOLA RAMOTA FROM RAHEEM A...,'000204813745,'061939204142,NIP,ECOBANK_20240619_085959,OUTWARDS
4,12209,ATM,'000010240619070556061956183389,Fund Transfer (Direct Credit),Approved or Completed Successfully,51000.0,2024-06-19 07:09:20,ECOBANK,ABUBAKAR OLANREWAJU AMBALI,FIRST BANK,AMBALI ABUBARKA OLANREWAJU,3051748163,TRANSFER TO AMBALI ABUBARKA OLANREWAJU FROM AB...,'000204814436,'061956183389,NIP,ECOBANK_20240619_085959,OUTWARDS


In [13]:
display(mobile_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,22,MOBILE PHONE,'000010240618230006111383882699,Fund Transfer (Direct Credit),Approved or Completed Successfully,10400.0,2024-06-19 00:00:11,ECOBANK,SOLOMON EFFIONG,ACCESS BANK,USORO UDUAK FRANCIS,29331278,REFNGA1400692133 BO SOLOMON EFFIONG IFO USORO UDU,'NGA1400692133,'111383882699,NIP,ECOBANK_20240619_055959,OUTWARDS
1,23,MOBILE PHONE,'000010240618230027111032687235,Fund Transfer (Direct Credit),Approved or Completed Successfully,15000.0,2024-06-19 00:00:30,ECOBANK,ndukwe ndukwe,ACCESS BANK,NNENNA NDUKWE,103834941,REFNGA1400692260 BO ndukwe ndukwe IFO NNENNA NDUK,'NGA1400692260,'111032687235,NIP,ECOBANK_20240619_055959,OUTWARDS
2,24,MOBILE PHONE,'000010240618230033111065196485,Fund Transfer (Direct Credit),Approved or Completed Successfully,112000.0,2024-06-19 00:00:33,ECOBANK,DAISI OLOYEDE,ACCESS BANK,CLEMENT CHIBUEZE OKEKE,1446522277,REF25005973944 BO DAISI OLOYEDE IFO CLEMENT CHIBU,'25005973944,'111065196485,NIP,ECOBANK_20240619_055959,OUTWARDS
3,25,MOBILE PHONE,'000010240618230033111218229745,Fund Transfer (Direct Credit),Approved or Completed Successfully,5000.0,2024-06-19 00:00:36,ECOBANK,OCHUKO IMIRHE,ACCESS BANK,GRACE CHAPEL,1686854062,REFNGA1400692300 BO OCHUKO IMIRHE IFO GRACE CHAPE,'NGA1400692300,'111218229745,NIP,ECOBANK_20240619_055959,OUTWARDS
4,26,MOBILE PHONE,'000010240618230107111366137289,Fund Transfer (Direct Credit),Approved or Completed Successfully,5000.0,2024-06-19 00:01:10,ECOBANK,okoi obeten,ACCESS BANK,COMFORT INI HANSON,1643471318,REFNGA1400692523 BO okoi obeten IFO COMFORT INI H,'NGA1400692523,'111366137289,NIP,ECOBANK_20240619_055959,OUTWARDS


In [14]:
display(agency_df.head())

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session,fileType
0,54,MOBILE PHONE,'000010240618231041111380830240,Fund Transfer (Direct Credit),Approved or Completed Successfully,13000.0,2024-06-19 00:10:41,ECOBANK,ADEFISAN EMMANUEL JSS1,ACCESS BANK,ADESOJI ABRAHAM OBAYOMI,1411272301,REFA241700103726169 BO ADEFISAN EMMANUEL JSS1 IFO,'A241700103726169,'111380830240,NIP,ECOBANK_20240619_055959,OUTWARDS
1,98,MOBILE PHONE,'000010240618232557111326567519,Fund Transfer (Direct Credit),Approved or Completed Successfully,5500000.0,2024-06-19 00:25:57,ECOBANK,KABIRU SHUAIBU DANSARAI CODE 0100000275,ACCESS BANK,HONEYWELL FLOUR MILLS PLC,1623646,REFA241700103726649 BO KABIRU SHUAIBU DANSARAI CO,'A241700103726649,'111326567519,NIP,ECOBANK_20240619_055959,OUTWARDS
2,112,MOBILE PHONE,'000010240618232952111164470842,Fund Transfer (Direct Credit),Approved or Completed Successfully,1050.0,2024-06-19 00:29:52,ECOBANK,Elizabeth Ajewole,ACCESS BANK,MUFUTAU ABDULLAHI,1513265498,REFA241700103726823 BO Elizabeth Ajewole IFO MUFU,'A241700103726823,'111164470842,NIP,ECOBANK_20240619_055959,OUTWARDS
3,133,MOBILE PHONE,'000010240618233640111335024813,Fund Transfer (Direct Credit),Approved or Completed Successfully,5000.0,2024-06-19 00:36:40,ECOBANK,Eboh Nonso Emmanuel,ACCESS BANK,VICTORIA CHINWENDU EBOH,98108739,REFA241700103726951 BO Eboh Nonso Emmanuel IFO VI,'A241700103726951,'111335024813,NIP,ECOBANK_20240619_055959,OUTWARDS
4,135,MOBILE PHONE,'000010240618233736111131210336,Fund Transfer (Direct Credit),Approved or Completed Successfully,125000.0,2024-06-19 00:37:36,ECOBANK,LAWAL,ACCESS BANK,LAWAN ABUBAKAR,1658062662,REFA241700103726957 BO LAWAL IFO LAWAN ABUBAKAR N,'A241700103726957,'111131210336,NIP,ECOBANK_20240619_055959,OUTWARDS


### Write Outputs to Output Directory

#### Define Output Directory and File Names

In [15]:
ebills_path = os.path.join(output_dir, f"EBILLS_{prev_day}.csv")
payable_path = os.path.join(output_dir, f"PAYABLE_{prev_day}.csv")
bulk_payable_path = os.path.join(output_dir, f"BULK PAYABLE_{prev_day}.csv")
receivable_path = os.path.join(output_dir, f"RECEIVABLE_{prev_day}.csv")
atm_path = os.path.join(output_dir, f"ATM_{prev_day}.csv")
mobile_path = os.path.join(output_dir, f"MOBILE_{prev_day}.csv")
agency_path = os.path.join(output_dir, f"AGENCY_{prev_day}.csv")

In [16]:
ebills_df.to_csv(ebills_path, index=False, mode='w')
print(f"Output file saved to: {ebills_path}")

payable_df.to_csv(payable_path, index=False, mode='w')
print(f"Output file saved to: {payable_path}")

bulk_payable_df.to_csv(bulk_payable_path, index=False, mode='w')
print(f"Output file saved to: {bulk_payable_path}")
      
receivable_df.to_csv(receivable_path, index=False, mode='w')
print(f"Output file saved to: {receivable_path}")

atm_df.to_csv(atm_path, index=False, mode='w')
print(f"Output file saved to: {atm_path}")

mobile_df.to_csv(mobile_path, index=False, mode='w')
print(f"Output file saved to: {mobile_path}")

agency_df.to_csv(agency_path, index=False, mode='w')
print(f"Output file saved to: {agency_path}")

Output file saved to: ../Output\EBILLS_19-06-2024.csv
Output file saved to: ../Output\PAYABLE_19-06-2024.csv
Output file saved to: ../Output\BULK PAYABLE_19-06-2024.csv
Output file saved to: ../Output\RECEIVABLE_19-06-2024.csv
Output file saved to: ../Output\ATM_19-06-2024.csv
Output file saved to: ../Output\MOBILE_19-06-2024.csv
Output file saved to: ../Output\AGENCY_19-06-2024.csv
